In [1]:
import dash
from dash import dcc, html, ctx
import requests

In [17]:
app = dash.Dash(__name__)

# define the layout of the dashboard
app.layout = html.Div( 
    children=[
        html.H1(children='OpenWeatherMap Dashboard', style={'text-align': 'center', 'background-color': '#333333', 'padding': '20px', 'color': 'white'}),
        html.Div(
            children=[
                dcc.Input(id='city-input', type='text', placeholder='Enter a city name', value='Jodhpur',
                          style={'margin-right': '10px', 'font-size': '20px', 'padding': '5px 10px', 'border': '1px solid #ddd', 'border-radius': '5px'}),
                html.Button(id='submit-button', n_clicks=0, children='Submit', style={'padding': '5px 10px', 'color': 'white', 'background-color': '#333333', 'border': 'none', 'border-radius': '5px', 'cursor': 'pointer', 'font-size': '16px', 'hover': 'background-color: #ddd'}),
                html.Button(id='change-unit',n_clicks=0 ,children='Change Unit', style={'padding': '5px 10px', 'color': 'white', 'background-color': '#424242', 'border': 'none', 'border-radius': '5px', 'cursor': 'pointer', 'font-size': '16px', 'hover': 'background-color: #ddd', 'margin-left':'5px'})
            ],
            style={'display': 'flex', 'justify-content': 'left'}
        ),
        html.Div(children=[ html.Div(id='main-weather', style={'flex': '50%','margin': '10px', 'font-size': '16px', 'padding': '5px 10px', 'border': '1px solid #ddd', 'border-radius': '5px'}),
                            html.Div(id='weather-table')], style={'display': 'flex', 'justify-content': 'left'}),
        dcc.Graph(id='forecast-graph')
    ],
    style={'max-width':'1200px','margin': '0 auto', 'font-family': 'Arial, sans-serif', 'background-color': '#f2f2f2', 'padding': '50px', 'text-align': 'center'}
)

# define a callback function to get latitude, longitude, and weather data from the OpenWeatherMap API
@app.callback(
    [dash.dependencies.Output('main-weather', 'children'),
     dash.dependencies.Output('weather-table', 'children'),
     dash.dependencies.Output('forecast-graph', 'figure')],
    [dash.dependencies.Input('submit-button', 'n_clicks')],
    [dash.dependencies.Input('change-unit', 'n_clicks')],
    [dash.dependencies.State('city-input', 'value')]
)
def update_output(n_clicks, btn, value):
    # make a request to the OpenWeatherMap Geocoding API to get the latitude and longitude of the input city
    # unit = 'metric' or unit = 'imperial'

    if btn % 2 == 0:
        unit = 'metric'
        speed_unit = 'mps'
        temp_unit = '°C'
    else:
        unit = 'imperial'
        speed_unit = 'mph'
        temp_unit = '°F'
        
    api_key = "25f3711d51ba44651a0369311124f949"
    geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={value}&limit=1&appid={api_key}"
    geo_response = requests.get(geo_url)
    geo_data = geo_response.json()
    lat = geo_data[0]['lat']
    lon = geo_data[0]['lon']
    
    # make a request to the OpenWeatherMap API to get the current weather data for the input city
    weather_url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units={unit}&appid={api_key}"
    weather_response = requests.get(weather_url)
    weather_data = weather_response.json()
    temperature = weather_data['main']['temp']
    feels_like = weather_data['main']['feels_like']
    humidity = weather_data['main']['humidity']
    wind_speed = weather_data['wind']['speed']
    min_temp = weather_data['main']['temp_min']
    max_temp = weather_data['main']['temp_max']
    pressure = weather_data['main']['pressure']
    
    description = weather_data['weather'][0]['description']
    
    
    # make a request to the OpenWeatherMap API to get the 5-day weather forecast for the input city
    forecast_url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&units={unit}&appid={api_key}"
    forecast_response = requests.get(forecast_url)
    forecast_data = forecast_response.json()
    
    # extract the date, temperature, and description for the next 5 days from the forecast data
    dates = []
    temperatures = []
    descriptions = []
    # print(weather_data)
    # print(len(forecast_data['list']))
    for i in range(0, len(forecast_data['list'])):
        date = forecast_data['list'][i]['dt_txt']
        temperature = forecast_data['list'][i]['main']['temp']
        description = forecast_data['list'][i]['weather'][0]['description']
        dates.append(date)
        temperatures.append(temperature)
        descriptions.append(description)
    
    
    # create a line chart using the extracted data and return it as the figure for the graph
    figure = {
        'data': [
            {'x': dates, 'y': temperatures, 'type': 'line', 'name': f'Temperature ({temp_unit})'},
        ],
        'layout': {
            'title': f'5-day Weather Forecast for {value}',
            'yaxis': {'title': f'Temperature ({temp_unit})'},
            'xaxis': {'title': 'Date'},
        }
    }
    table_style = {
        'border': '1px solid #ddd',
        'border-collapse': 'collapse',
        'margin': '50px',
        'font-size': '1.2em',
    }

    row_style = {
        'border': '1px solid #ddd',
        'padding': '10px',
        'text-align': 'left',
        'margin': '20px',
    }
    # I want to show this left side of the page
    main_weather = html.Div(children=[
        html.H3(style={'color':'#434343'},children=f"{value}"),
        html.H1(children=f"{temperature}{temp_unit}"),
        html.H2(style={'color':'#434343'}, children=f"Feels Like: {feels_like}{temp_unit}"),
        html.H3(style={'color':'#434343'}, children=f"Weather: {description}"),
    ], style={'margin-bottom': '20px', 'align': 'left'})
    
    output = html.Div(children=[
        html.Table(style=table_style, children=[
            html.Tr(children=[
                html.Td("Humidity"),
                html.Td(f"{humidity}%")
            ], style=row_style),
            html.Tr(children=[
                html.Td("Wind speed"),
                html.Td(f"{wind_speed} {speed_unit}")
            ], style=row_style),
            html.Tr(children=[
                html.Td("Min temperature"),
                html.Td(f"{min_temp}{temp_unit}")
            ], style=row_style),
            html.Tr(children=[
                html.Td("Max temperature"),
                html.Td(f"{max_temp}{temp_unit}")
            ], style=row_style),
            html.Tr(children=[
                html.Td("Pressure"),
                html.Td(f"{pressure} hPa")
            ], style=row_style)
        ])
    ])
    return main_weather, output, figure

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [26/Apr/2023 03:23:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 03:23:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 03:23:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 03:23:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2023 03:23:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2023 03:23:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 03:24:08] "POST /_dash-update-component HTTP/1.1" 200 -
